<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/Algorithm_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
plt_kwargs = {'figsize': (10, 4)}
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier


In [2]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount(r'drive/folders/0AAxMXLC9WPNxUk9PVA')


Mounted at /content/drive


In [3]:
ls

drive/  sample_data/


In [4]:
cd 'drive/Shared drives/CS_GO_Data/processed_data_v2'

/content/drive/Shared drives/CS_GO_Data/processed_data_v2


In [5]:
ls

aggregated_data/  de_nuke.png       mm_grenades_demos.csv
cs_cruise.png     de_overpass.png   out-de_cache.png
damage_data.csv   de_royal.png      out-de_cbble.png
de_cache.png      de_santorini.png  out-de_dust2.png
de_cbble.png      de_season.png     out-de_inferno.png
de_coast.png      de_train.png      out-de_mirage.png
de_dust2.png      de_tulip.png      out-de_overpass.png
de_empire.png     grenade_data.csv  out-de_train.png
de_inferno.png    kill_data.csv     winnerSide_matches.csv
de_mikla.png      Log_ROC.png       winnerSide_rounds.csv
de_mirage.png     map_data.csv      winnerSide_teams.csv
de_new_nuke.png   meta_data.csv


In [6]:
df_winner_matches_agg = pd.read_csv("aggregated_data/modelinput/winner_matches_agg.csv")

df_winner_side_rounds_agg = pd.read_csv("aggregated_data/modelinput/winner_side_rounds_agg.csv")

df_winner_team_rounds_agg = pd.read_csv("aggregated_data/modelinput/winner_team_rounds_agg.csv")


# Data Preprocess for Data Mining Algorithms


In [7]:
df_esea_meta_demos =  pd.read_csv("meta_data.csv")

df_esea_meta_demos.head()

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550


In [11]:
def terrorist_counterT_win(series_shifted_winner_side):
  terrorist_win = series_shifted_winner_side.apply(lambda y: 1 if y=='Terrorist' else 0)
  terrorist_win = terrorist_win.iloc[:-1]
  terrorist_win = terrorist_win.reset_index(drop=True)

  counterterrorist_win = series_shifted_winner_side.apply(lambda y: 1 if y=='CounterTerrorist' else 0)
  counterterrorist_win = counterterrorist_win.iloc[:-1]
  counterterrorist_win = counterterrorist_win.reset_index(drop=True)

  return terrorist_win, counterterrorist_win

def preprocess_meta_demos_algo(df_esea_meta_demos):

  df_original_preprocessed = df_esea_meta_demos[['round','ct_eq_val','t_eq_val']]

  unique_maps = df_esea_meta_demos["map"].unique()

  map_dataframe = pd.DataFrame()

  for map_cs_go in unique_maps:
    map_dataframe[map_cs_go] = df_esea_meta_demos["map"].apply(lambda y: 1 if y == map_cs_go else 0)

  series_one_element = pd.Series([0])

  binary_values_first_round = df_esea_meta_demos["round"].apply(lambda y : 0 if y == 1 else 1)

  binary_values_first_round = binary_values_first_round.rename("not_First_Round").reset_index(drop=True)

  series_shifted_winner_side = series_one_element.append(df_esea_meta_demos["winner_side"])

  terrorist_win, counterterrorist_win = terrorist_counterT_win(series_shifted_winner_side)

  terrorist_win = (terrorist_win * binary_values_first_round).rename('terrorist_win')

  counterterrorist_win = (counterterrorist_win * binary_values_first_round).rename('counterterrorist_win')

  previous_win_df = pd.concat([terrorist_win, counterterrorist_win])

  data_to_predict_T_CT = df_esea_meta_demos['winner_side'].apply(lambda y: 1 if y=='Terrorist' else 0)

  column_names = [df_original_preprocessed, map_dataframe, terrorist_win, counterterrorist_win, data_to_predict_T_CT]

  return pd.concat(column_names, axis = 'columns', join = 'inner')

In [12]:
df_meta_data = preprocess_meta_demos_algo(df_esea_meta_demos)

agg_df_rounds = pd.concat([df_winner_side_rounds_agg, df_meta_data[['terrorist_win', 'counterterrorist_win',]]], axis=1)

agg_df_rounds



,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_side_class,terrorist_win,counterterrorist_win
0,1,94.30782,160.9591,66.65128,4300,4250,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0
1,2,160.95910,279.3998,118.44070,6300,19400,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1,1,0
2,3,279.39980,341.0084,61.60860,7650,19250,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1,1,0
3,4,341.00840,435.4259,94.41750,24900,23400,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1,1,0
4,5,435.42590,484.2398,48.81390,5400,20550,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,-1,1,0
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,-1,1,0
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,-1,1,0
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,-1,1,0


In [ ]:
def plot_hist(df):
  arr = ['LogReg', 'RF', 'KNN', 'SVM', "XGB"]
  for i, col in enumerate(df.columns):
    plt.figure(i)
    sns.distplot(df[col])
  plt.xlabel(["fit_time_average","score_time_average","test_accuracy_average","test_precision_weighted_average","test_recall_weighted_average","test_f1_weighted_average", "test_roc_auc_average"])


# Models for WinnerSide based on Rounds

In [68]:
agg_df_rounds = agg_df_rounds[agg_df_rounds["winner_side_class"] != 0]

X = agg_df_rounds.drop('winner_side_class', axis = 1)[:5000]

y = agg_df_rounds['winner_side_class'][:5000]


# using train_test_split for splitting the data into four different sets with 20% for the test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale the data
standard_scalar = StandardScaler()

X_train = standard_scalar.fit_transform(X_train)

X_test = standard_scalar.fit_transform(X_test)

model_eval = [
            ('LogReg', LogisticRegression()), 
            ('RF', RandomForestClassifier(max_depth=2, random_state=0)),
            ('KNN', KNeighborsClassifier(n_neighbors=8)),
            ('SVM', svm.SVC(kernel='linear', C=0.2)),
            ("XGB", XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100)),
            ("ADA", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',max_depth=3, 
                                                                             max_features=None, max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0, 
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1, 
                                                                             min_samples_split=2,
                                                                             min_weight_fraction_leaf=0.0, 
                                                                             presort='deprecated', 
                                                                             random_state=None, 
                                                                             splitter='best'), 
                                                                              learning_rate=0.2, n_estimators=50))]

In [69]:
def run_models_winner_side_rounds(X_train, X_test, Y_train, Y_test, model_eval):
  scoring_metrics = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
  df_to_output = []  
  results = []
  names = []
  for name, model in model_eval:
    kfold_validation = model_selection.KFold(n_splits=6, shuffle=True)
    cv_result_kfold = model_selection.cross_validate(model, X_train, Y_train, cv=kfold_validation, scoring=scoring_metrics)
    model_fit = model.fit(X_train, Y_train)
    y_pred = model_fit.predict(X_test)
    print(name)
    print(classification_report(Y_test, y_pred))
    results.append(cv_result_kfold)
    df_to_append = pd.DataFrame(cv_result_kfold)
    df_to_append['model'] = name
    df_to_output.append(df_to_append)
  final = pd.concat(df_to_output, ignore_index=True)
  return final

In [70]:
df_kfold_results = run_models_winner_side_rounds(X_train, X_test, Y_train, Y_test, model_eval)

df_kfold_results_average_values = df_kfold_results.groupby('model').mean()

df_kfold_results_average_values.columns = [str(col) + '_average' for col in df_kfold_results_average_values.columns]

df_kfold_results_average_values

# df_kfold_results_average_values.to_csv("winnerSide_rounds.csv")

LogReg
              precision    recall  f1-score   support

          -1       0.76      0.77      0.76       505
           1       0.76      0.75      0.75       495

    accuracy                           0.76      1000
   macro avg       0.76      0.76      0.76      1000
weighted avg       0.76      0.76      0.76      1000

RF
              precision    recall  f1-score   support

          -1       0.75      0.80      0.77       505
           1       0.78      0.73      0.75       495

    accuracy                           0.76      1000
   macro avg       0.76      0.76      0.76      1000
weighted avg       0.76      0.76      0.76      1000

KNN
              precision    recall  f1-score   support

          -1       0.72      0.79      0.75       505
           1       0.76      0.69      0.72       495

    accuracy                           0.74      1000
   macro avg       0.74      0.74      0.74      1000
weighted avg       0.74      0.74      0.74      1000

SVM
 

,fit_time_average,score_time_average,test_accuracy_average,test_precision_weighted_average,test_recall_weighted_average,test_f1_weighted_average,test_roc_auc_average
model,,,,,,,
ADA,0.570907,0.031604,0.776008,0.777537,0.776008,0.775878,0.831474
KNN,0.011261,0.221424,0.742491,0.746510,0.742491,0.741647,0.801628
LogReg,0.038604,0.007753,0.778253,0.779028,0.778253,0.778144,0.844114
RF,0.228653,0.028261,0.777253,0.779888,0.777253,0.776664,0.834777
SVM,0.481406,0.074402,0.770001,0.774287,0.770001,0.769009,0.833850
XGB,0.425725,0.011632,0.781502,0.781977,0.781502,0.781357,0.848627


# Models for WinnerSide based on Matches

In [65]:
def run_models_multiclass(X_train, X_test, Y_train, Y_test, model_eval):
  scoring_metrics = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
  df_to_output = []  
  results = []
  names = []
  for name, model in model_eval:
    kfold_validation = model_selection.KFold(n_splits=6, shuffle=True)
    cv_result_kfold = model_selection.cross_validate(model, X_train, Y_train, cv=kfold_validation, scoring=scoring_metrics)
    model_fit = model.fit(X_train, Y_train)
    y_pred = model_fit.predict(X_test)
    print(name)
    print(classification_report(Y_test, y_pred))
    results.append(cv_result_kfold)
    df_to_append = pd.DataFrame(cv_result_kfold)
    df_to_append['model'] = name
    df_to_output.append(df_to_append)
  final = pd.concat(df_to_output, ignore_index=True)
  return final

In [66]:
df_winner_matches_agg = df_winner_matches_agg.dropna()

X = df_winner_matches_agg.drop('winner_class', axis = 1)[:5000]

y = df_winner_matches_agg['winner_class'][:5000]


# using train_test_split for splitting the data into four different sets with 20% for the test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale the data
standard_scalar = StandardScaler()

X_train = standard_scalar.fit_transform(X_train)

X_test = standard_scalar.fit_transform(X_test)

models_eval = [
            ('LogReg', LogisticRegression(multi_class="ovr")),
            ('RF', RandomForestClassifier(max_depth=2, random_state=0)),
            ('KNN', KNeighborsClassifier(n_neighbors=8)),
            ('SVM', svm.SVC(kernel='linear', C=1)),
            ("XGB", XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100, objective = 'multi:softmax')),
            ("ADA", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',max_depth=3, 
                                                                             max_features=None, max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0, 
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1, 
                                                                             min_samples_split=2,
                                                                             min_weight_fraction_leaf=0.0, 
                                                                             presort='deprecated', 
                                                                             random_state=None, 
                                                                             splitter='best'), 
                                                                              learning_rate=0.5, n_estimators=10))]

In [67]:
df_kfold_results = run_models_multiclass(X_train, X_test, Y_train, Y_test, models_eval)

df_kfold_results_average_values = df_kfold_results.groupby('model').mean()

df_kfold_results_average_values.columns = [str(col) + '_average' for col in df_kfold_results_average_values.columns]

df_kfold_results_average_values

# df_kfold_results_average_values.to_csv("winnerSide_matches.csv")

LogReg
              precision    recall  f1-score   support

          -1       0.46      0.42      0.44       463
           0       0.00      0.00      0.00        45
           1       0.52      0.62      0.57       492

    accuracy                           0.50      1000
   macro avg       0.33      0.34      0.34      1000
weighted avg       0.47      0.50      0.48      1000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

RF
              precision    recall  f1-score   support

          -1       0.44      0.26      0.33       463
           0       0.00      0.00      0.00        45
           1       0.53      0.78      0.63       492

    accuracy                           0.51      1000
   macro avg       0.32      0.35      0.32      1000
weighted avg       0.47      0.51      0.46      1000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNN
              precision    recall  f1-score   support

          -1       0.47      0.59      0.53       463
           0       0.33      0.09      0.14        45
           1       0.50      0.42      0.46       492

    accuracy                           0.48      1000
   macro avg       0.44      0.37      0.37      1000
weighted avg       0.48      0.48      0.47      1000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

SVM
              precision    recall  f1-score   support

          -1       0.46      0.48      0.47       463
           0       0.00      0.00      0.00        45
           1       0.52      0.55      0.53       492

    accuracy                           0.49      1000
   macro avg       0.33      0.34      0.33      1000
weighted avg       0.47      0.49      0.48      1000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB
              precision    recall  f1-score   support

          -1       0.47      0.41      0.44       463
           0       0.29      0.24      0.27        45
           1       0.53      0.60      0.56       492

    accuracy                           0.50      1000
   macro avg       0.43      0.42      0.42      1000
weighted avg       0.49      0.50      0.49      1000

ADA
              precision    recall  f1-score   support

          -1       0.48      0.47      0.47       463
           0       0.33      0.29      0.31        45
           1       0.54      0.55      0.55       492

    accuracy                           0.50      1000
   macro avg       0.45      0.44      0.44      1000
weighted avg       0.50      0.50      0.50      1000



,fit_time_average,score_time_average,test_accuracy_average,test_precision_weighted_average,test_recall_weighted_average,test_f1_weighted_average
model,,,,,,
ADA,0.058394,0.006498,0.495002,0.491228,0.495002,0.489410
KNN,0.013133,0.050800,0.484248,0.484990,0.484248,0.476644
LogReg,0.034589,0.004916,0.510501,0.494510,0.510501,0.498383
RF,0.202905,0.018451,0.522246,0.493154,0.522246,0.473379
SVM,0.421820,0.034437,0.504742,0.485034,0.504742,0.493881
XGB,0.586779,0.011122,0.507762,0.508710,0.507762,0.504721


# Models for WinnerSide based on Teams

In [61]:
X = df_winner_team_rounds_agg.drop('winner_class', axis = 1)[:5000]

y = df_winner_team_rounds_agg['winner_class'][:5000]

# using train_test_split for splitting the data into four different sets with 20% for the test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale the data
standard_scalar = StandardScaler()

X_train = standard_scalar.fit_transform(X_train)

X_test = standard_scalar.fit_transform(X_test)

models_eval = [
            ('LogReg', LogisticRegression()),
            ('RF', RandomForestClassifier(max_depth=2, random_state=0)),
            ('KNN', KNeighborsClassifier(n_neighbors=8)),
            ('SVM', svm.SVC(kernel='linear', C=1)),
            ("XGB", XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100)),
            ("ADA", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',max_depth=3, 
                                                                             max_features=None, max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0, 
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1, 
                                                                             min_samples_split=2,
                                                                             min_weight_fraction_leaf=0.0, 
                                                                             presort='deprecated', 
                                                                             random_state=None, 
                                                                             splitter='best'), 
                                                                            learning_rate=0.5, n_estimators=10))]

In [62]:
def run_models_winnerside_team(X_train, X_test, Y_train, Y_test, models_eval):
  scoring_metrics = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
  df_to_output = []  
  results = []
  names = []
  for name, model in models_eval:
    kfold_validation = model_selection.KFold(n_splits=6, shuffle=True)
    cv_result_kfold = model_selection.cross_validate(model, X_train, Y_train, cv=kfold_validation, scoring=scoring_metrics)
    model_fit = model.fit(X_train, Y_train)
    y_pred = model_fit.predict(X_test)
    print(name)
    print(classification_report(Y_test, y_pred))
    results.append(cv_result_kfold)
    df_to_append = pd.DataFrame(cv_result_kfold)
    df_to_append['model'] = name
    df_to_output.append(df_to_append)
  final = pd.concat(df_to_output, ignore_index=True)
  return final

In [63]:
df_kfold_results = run_models_winnerside_team(X_train, X_test, Y_train, Y_test, models_eval)

df_kfold_results_average_values = df_kfold_results.groupby('model').mean()

df_kfold_results_average_values.columns = [str(col) + '_average' for col in df_kfold_results_average_values.columns]

df_kfold_results_average_values
# df_kfold_results_average_values.to_csv("winnerSide_teams.csv")

LogReg
              precision    recall  f1-score   support

          -1       0.56      0.58      0.57       511
           1       0.55      0.53      0.54       489

    accuracy                           0.55      1000
   macro avg       0.55      0.55      0.55      1000
weighted avg       0.55      0.55      0.55      1000

RF
              precision    recall  f1-score   support

          -1       0.72      0.79      0.75       511
           1       0.75      0.67      0.71       489

    accuracy                           0.73      1000
   macro avg       0.73      0.73      0.73      1000
weighted avg       0.73      0.73      0.73      1000

KNN
              precision    recall  f1-score   support

          -1       0.73      0.75      0.74       511
           1       0.73      0.71      0.72       489

    accuracy                           0.73      1000
   macro avg       0.73      0.73      0.73      1000
weighted avg       0.73      0.73      0.73      1000

SVM
 

,fit_time_average,score_time_average,test_accuracy_average,test_precision_weighted_average,test_recall_weighted_average,test_f1_weighted_average,test_roc_auc_average
model,,,,,,,
ADA,0.116034,0.010475,0.755241,0.755923,0.755241,0.755192,0.807716
KNN,0.010649,0.206996,0.733249,0.734817,0.733249,0.732756,0.798038
LogReg,0.025814,0.008806,0.532253,0.533038,0.532253,0.532084,0.535253
RF,0.229853,0.025843,0.695742,0.707723,0.695742,0.691678,0.777316
SVM,0.736850,0.124019,0.537251,0.537917,0.537251,0.537000,0.545260
XGB,0.396771,0.012059,0.765998,0.766432,0.765998,0.766057,0.830579
